In [59]:
import torch
import numpy as np
import sys
import os
sys.path.insert(0, os.path.abspath('../'))
from DNN_module import Net

%run ../NCoinDP_functions.ipynb
torch.set_default_device("cpu")

In [ ]:
# Synthetic data for uncertainty
tmp = "infer_sim_0.pt" # uniform prior
tmp = torch.load(tmp)
X = tmp[0]
Y = tmp[1]

# test data, but wanted to extract just a single (first) row
n = 3000
scenarios = ["S1", "S2", "S3"]
test_save_name = '../test_data/OU_test_n'+ str(n) + '_' + scenarios[2] +'.pt'
test = torch.load(test_save_name)
torch.set_default_device("cpu")

# True parameter
print(test[1][1])

x0 = OU_summary(test[0][1])[0,:]
x0 = torch.reshape(x0, (1,5))

uncertainty_list = []
r_list = [.05 ,.07, .10]
for r in r_list:
    for sim in range(20):
        X = X.to("cpu")
        Y = Y.to("cpu")
        x0 = x0.to("cpu")

        L = X.size()[0]
        L_train = 250000
        L_val = 50000
        L_cal = L - L_train - L_val

        # Divide Data
        X_cal = X[(L_train + L_val):].to("cpu")
        Y_cal = Y[(L_train + L_val):].to("cpu")

        model_save_name = str(sim) +'.pt'
        path = F"infer_nets/net0/{model_save_name}"
        torch.set_default_device("cpu")

        # load net
        net = Net(D_in = 5, D_out = 3, H = 256, H2 = 256, H3 = 256)
        net.load_state_dict(torch.load(path))
        net.eval()

        tmp = conf_inf_sd(x0, X_cal, Y_cal, net, r) # .05, .07, .10

        uncertainty_list.append(tmp)
        print("====================", "sim: ", sim, "=====================" )
        torch.save(uncertainty_list, "uncertainty_" +str(r)+".pt") # .05, .07, .10

[3, 2, 1.0]
train_loss: 0.227071  [   64/53900]
train_loss: 0.198267  [ 6464/53900]
train_loss: 0.203991  [12864/53900]
train_loss: 0.193166  [19264/53900]
train_loss: 0.207502  [25664/53900]
train_loss: 0.200242  [32064/53900]
train_loss: 0.195966  [38464/53900]
train_loss: 0.217536  [44864/53900]
train_loss: 0.203832  [51264/53900]
Epoch 1
-------------------------------
train_loss 0.203549 val_loss 0.203761
Epoch 31
-------------------------------
train_loss 0.023495 val_loss 0.023543


In [60]:
import pandas as pd
tmp = torch.load("uncertainty_10.pt")
print(len(tmp))
print(".10")


cvrg_results = np.zeros( (len(tmp),3,3 ))
cvrg_results.shape

# Read a CSV file into a DataFrame
df = pd.read_csv('MCMC/MC_results.txt', sep="\t")
MC_results = torch.tensor(df.values)

param_name = ["kappa", "beta", "sigma2"]
coverage = [0.85, 0.90, 0.95]

for k in range(len(tmp)):
    for i in range(3):
        values = tmp[k][1][:,i].detach().cpu().numpy().tolist()
        weights = tmp[k][0].numpy()
        for j in range(3):
            error_cvrg = 1-coverage[j]
            interval = [error_cvrg/2, 1 - error_cvrg/2]
            intvl = weighted_quantile(values, interval, weights)

            cvrg = torch.sum((MC_results[:,i] < intvl[1]) & (intvl[0] < MC_results[:,i]))/MC_results.size()[0]
            
            cvrg_results[k,i,j] = cvrg.detach().cpu().numpy().tolist()
            #print("coverage: ", coverage[j], param_name[i], "inf_m: ", cov_m)
            #print("coverage: ", coverage[j], param_name[i],"inf_sd: ", cvrg)
for i in range(3):
    for j in range(3):
        #print("coverage: ", coverage[j], "param: ", param_name[i], round(np.mean(cvrg_results[:,i,j]),3), "sd: ", round(np.std(cvrg_results[:,i,j]),3))
        x = cvrg_results[:,i,j]
        print("coverage: ", coverage[j], "param: ", param_name[i], round(np.median(x),3), "mad: ", round(np.median(np.absolute(x - np.median(x))),3))

3
.10
coverage:  0.85 param:  kappa 0.833 mad:  0.0
coverage:  0.9 param:  kappa 0.883 mad:  0.004
coverage:  0.95 param:  kappa 0.937 mad:  0.004
coverage:  0.85 param:  beta 0.851 mad:  0.005
coverage:  0.9 param:  beta 0.902 mad:  0.002
coverage:  0.95 param:  beta 0.947 mad:  0.003
coverage:  0.85 param:  sigma2 0.854 mad:  0.004
coverage:  0.9 param:  sigma2 0.904 mad:  0.003
coverage:  0.95 param:  sigma2 0.954 mad:  0.001


In [ ]:
math.inf == math.inf